## MediaEval Video memorability - predicting long term and short term video memorability scores

In [ ]:
# import statements

import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn import linear_model
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline
from scipy.stats import spearmanr
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

In [57]:
import os
os.getcwd()
#drive.mount('/content/drive/', force_remount=True)
#os.chdir(r'C:\Users\navee\Desktop\Dev-set')
#os.getcwd()

'C:\\Users\\navee\\Desktop\\Dev-set'

In [58]:
# load the ground truth values
#label_path = './Ground-truth/'
label_path = './Dev-set/Ground-truth/'
ground_truth=pd.read_csv(label_path+'ground-truth.csv')
ground_truth.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


Checking the correlation between the columns

In [59]:
ground_truth.corr()

,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
short-term_memorability,1.000000,-0.021178,0.325002,-0.013858
nb_short-term_annotations,-0.021178,1.000000,-0.047093,0.285548
long-term_memorability,0.325002,-0.047093,1.000000,0.036051
nb_long-term_annotations,-0.013858,0.285548,0.036051,1.000000


In [60]:
ground_truth = ground_truth.drop(['nb_short-term_annotations', 'nb_long-term_annotations'], axis=1)
ground_truth.head()

,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900


getting the captions of the videos from the data set

In [61]:
captions_dataset = pd.read_csv('./Dev-set/Captions/dev-set_video-captions.txt', sep = '\t', header=None, names=['video', 'caption'] )
captions_dataset.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


merging the captions and the ground truth

In [62]:
gt_caption_merge = ground_truth.merge(captions_dataset)
gt_caption_merge.head()

,video,short-term_memorability,long-term_memorability,caption
0,video3.webm,0.924,0.846,blonde-woman-is-massaged-tilt-down
1,video4.webm,0.923,0.667,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,0.863,0.700,khr-gangsters
3,video8.webm,0.922,0.818,medical-helicopter-hovers-at-airport
4,video10.webm,0.950,0.900,couple-relaxing-on-picnic-crane-shot


cleaning the captions data, i,e. the text data has to be pre processed and cleaned before implementing any sort of text based model. Removing the special characters and stop words. Lemmatizing in order to get the base meaning of the words too.

In [63]:
#cleaning the caption data.

#Converting the captions into lowercase
gt_caption_merge['caption'] = gt_caption_merge['caption'].str.replace('-',' ')

#Removal of punctuations 
gt_caption_merge['caption'] = gt_caption_merge['caption'].str.replace('[^\w\s]','')

#Removal of stop words
words = stopwords.words('english')
gt_caption_merge['caption'] = gt_caption_merge['caption'].apply(lambda x: " ".join(each for each in x.split() if each not in words))
#gt_caption_c3d_merge['caption'].head()

#Lemmatizing our captions 
lemmatizer = WordNetLemmatizer() 
gt_caption_merge['caption'] = gt_caption_merge['caption'].apply(lambda x: " ".join([lemmatizer.lemmatize(each) for each in x.split()]))
#gt_caption_c3d_merge['caption'].head()

#removing digits from captions
gt_caption_merge['caption'] = gt_caption_merge['caption'].str.replace('[0-9]','')
gt_caption_merge.head()

,video,short-term_memorability,long-term_memorability,caption
0,video3.webm,0.924,0.846,blonde woman massaged tilt
1,video4.webm,0.923,0.667,roulette table spinning ball closeup shot
2,video6.webm,0.863,0.700,khr gangster
3,video8.webm,0.922,0.818,medical helicopter hovers airport
4,video10.webm,0.950,0.900,couple relaxing picnic crane shot


choosing the C3D feature of the video features available, below is the part for extracting the C3D feature and merging with the ground_caption_merge data frame

In [64]:
#function to read C3D feature
def c3d_read(df, filename):
  df1 = pd.read_csv(filename, header=None, sep=' ')
  df1['video'] = filename[filename.find('video'):filename.find('.txt')] + '.webm'
  df = df.append(df1, ignore_index=True)
  return df

#read file names from the path mentioned
def reading_file_names(path):
  files = []
  file_number=0
  # r=root, d=directories, f = files
  for r, d, f in os.walk(path):
      for file in f:
          if '.txt' in file:
              files.append(os.path.join(r, file))
              file_number = file_number + 1
  print("total files read = " + str(file_number))
  return files

#read c3d train data
path=r'./Dev-set/C3D' #(need to chnage the path as per the directory the of C3D)
files = reading_file_names(path)
dataframe_C3D = pd.DataFrame()
for filename in files:
    dataframe_C3D = c3d_read(dataframe_C3D, filename)
    
dataframe_C3D.head()

total files read = 6000


,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,video
0,0.000090,0.000615,0.003436,0.001281,3.551400e-03,3.128800e-04,0.000042,0.000018,0.000157,1.171300e-04,...,1.708900e-04,0.000231,6.371500e-04,4.001000e-05,6.142000e-05,0.000075,0.000002,0.001323,NaN,video10.webm
1,0.002312,0.009966,0.027439,0.000006,2.155300e-04,1.394600e-04,0.000111,0.000029,0.000046,2.388820e-03,...,3.939800e-04,0.008439,1.255300e-04,1.976800e-04,1.048400e-04,0.075047,0.287573,0.003510,NaN,video100.webm
2,0.468035,0.489501,0.000079,0.000010,2.000000e-07,1.540000e-06,0.000168,0.000010,0.000034,5.200000e-07,...,7.000000e-08,0.000006,1.790000e-06,9.200000e-07,1.950000e-06,0.000127,0.000714,0.000408,NaN,video1001.webm
3,0.014036,0.000492,0.000233,0.000028,6.320000e-06,1.330000e-06,0.000006,0.000002,0.000007,4.002000e-05,...,1.878000e-05,0.000049,3.068000e-05,1.310000e-06,7.410000e-06,0.830456,0.002719,0.001101,NaN,video1003.webm
4,0.046734,0.000868,0.000005,0.000008,2.000000e-08,5.600000e-07,0.000013,0.000004,0.000001,5.990000e-06,...,4.500000e-07,0.000004,2.400000e-07,1.100000e-07,3.300000e-07,0.000002,0.000018,0.000080,NaN,video1004.webm


there is a nan value column (column number 101) in the dataframe_C3D, so dropping it

In [65]:
# dropiing the nan value column 
dataframe_C3D = dataframe_C3D.drop(columns=101)
dataframe_C3D.shape

(6000, 102)

changing the column names of dataframe_C3D dataframe for better interpretation

In [66]:
new_columns = list()
for colname in dataframe_C3D.columns:
    new_columns.append('C3D_' + str(colname))
dataframe_C3D.columns = new_columns
dataframe_C3D = dataframe_C3D.rename(columns={'C3D_video':'video'})
dataframe_C3D.head()

,C3D_0,C3D_1,C3D_2,C3D_3,C3D_4,C3D_5,C3D_6,C3D_7,C3D_8,C3D_9,...,C3D_92,C3D_93,C3D_94,C3D_95,C3D_96,C3D_97,C3D_98,C3D_99,C3D_100,video
0,0.000090,0.000615,0.003436,0.001281,3.551400e-03,3.128800e-04,0.000042,0.000018,0.000157,1.171300e-04,...,6.900100e-04,1.708900e-04,0.000231,6.371500e-04,4.001000e-05,6.142000e-05,0.000075,0.000002,0.001323,video10.webm
1,0.002312,0.009966,0.027439,0.000006,2.155300e-04,1.394600e-04,0.000111,0.000029,0.000046,2.388820e-03,...,1.226390e-03,3.939800e-04,0.008439,1.255300e-04,1.976800e-04,1.048400e-04,0.075047,0.287573,0.003510,video100.webm
2,0.468035,0.489501,0.000079,0.000010,2.000000e-07,1.540000e-06,0.000168,0.000010,0.000034,5.200000e-07,...,1.300000e-07,7.000000e-08,0.000006,1.790000e-06,9.200000e-07,1.950000e-06,0.000127,0.000714,0.000408,video1001.webm
3,0.014036,0.000492,0.000233,0.000028,6.320000e-06,1.330000e-06,0.000006,0.000002,0.000007,4.002000e-05,...,4.673000e-05,1.878000e-05,0.000049,3.068000e-05,1.310000e-06,7.410000e-06,0.830456,0.002719,0.001101,video1003.webm
4,0.046734,0.000868,0.000005,0.000008,2.000000e-08,5.600000e-07,0.000013,0.000004,0.000001,5.990000e-06,...,1.000000e-08,4.500000e-07,0.000004,2.400000e-07,1.100000e-07,3.300000e-07,0.000002,0.000018,0.000080,video1004.webm


In [67]:
#merging dataframe_C3D dataframe with gt_caption_merge dataframe
gt_caption_c3d_merge = gt_caption_merge.merge(dataframe_C3D)
gt_caption_c3d_merge.head()

,video,short-term_memorability,long-term_memorability,caption,C3D_0,C3D_1,C3D_2,C3D_3,C3D_4,C3D_5,...,C3D_91,C3D_92,C3D_93,C3D_94,C3D_95,C3D_96,C3D_97,C3D_98,C3D_99,C3D_100
0,video3.webm,0.924,0.846,blonde woman massaged tilt,0.020249,0.001578,0.000826,0.000945,0.000063,0.000003,...,0.001042,0.000161,0.000257,0.046617,0.000156,0.000006,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.923,0.667,roulette table spinning ball closeup shot,0.000118,0.000891,0.000188,0.000045,0.000063,0.000002,...,0.000582,0.000393,0.000864,0.000947,0.000136,0.000007,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.863,0.700,khr gangster,0.011765,0.000746,0.000784,0.000013,0.000007,0.000028,...,0.000224,0.000003,0.000031,0.002538,0.000104,0.000005,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.922,0.818,medical helicopter hovers airport,0.000223,0.000165,0.000007,0.000016,0.000005,0.000014,...,0.000046,0.000009,0.000023,0.000053,0.000048,0.000019,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.950,0.900,couple relaxing picnic crane shot,0.000090,0.000615,0.003436,0.001281,0.003551,0.000313,...,0.000037,0.000690,0.000171,0.000231,0.000637,0.000040,0.000061,0.000075,0.000002,0.001323


since we want to determine the memorablity scores which is a continous value between 0 and 1, lets fix this as a regression problem. Lets explore few of sklearns regression models to find out the best fit one and proceed with it 

In [68]:
#seperating the lables and the feratures from the data frame
training_set =  gt_caption_c3d_merge.drop(['short-term_memorability','long-term_memorability'], axis=1)
testing_set_short_term = gt_caption_c3d_merge['short-term_memorability']
testing_set_long_term = gt_caption_c3d_merge['long-term_memorability']

In [69]:
#function to get the spearnman's correlation values for the models run
def spearman_corr(model, traing_caption, test_caption, y_train, y_test):
    model.fit(traing_caption, y_train)
    predictions_validation = model.predict(test_caption)
    corr, p_value = spearmanr(y_test, predictions_validation)
    return corr
   # return predictions_validation

In [70]:
#tfidf vectorizer for captions
def vectorize_data(X_training, X_test):
    bow = TfidfVectorizer(lowercase=True, ngram_range=(1,2),analyzer = "word")
    train_bow = bow.fit(X_training['caption'])
    training_caption = train_bow.transform(X_training['caption'])
    test_caption = train_bow.transform(X_test['caption'])
    return training_caption, test_caption

# Working on captions feature alone

In [ ]:
#calculationg scores for long term and short term scores for captions feature using regression models of sklearn like 
#bayseian ridge 
#linear regression 
#GradientBoostingRegressor
#RandomForestRegressor
#stackingRegressor (ensemble model with base estimators and one final estimator)
#Ridge regressor

#caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#base estimators for the stacking regressor model with the fianl estimator being Linear regression
#estimators = [('gbr', GradientBoostingRegressor(random_state=1, n_estimators=10)),('rfr', RandomForestRegressor(random_state=1, n_estimators=10)),('br', BayesianRidge())]

# #long_term
# X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)

# training_caption, testing_caption  = vectorize_data(X_train, X_test)

# long_term_score_br = spearman_corr(BayesianRidge(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# long_term_score_lr = spearman_corr(LinearRegression(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# long_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# long_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=500), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# long_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# long_term_score_rlr = spearman_corr(Ridge(alpha=0.5), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# #short_term
# X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)

# training_caption, testing_caption  = vectorize_data(X_train, X_test)

# short_term_score_br = spearman_corr(BayesianRidge(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# short_term_score_lr = spearman_corr(LinearRegression(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# short_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# short_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=500), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# short_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# short_term_score_rlr = spearman_corr(Ridge(alpha=0.5), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)

# print("long term score for caption bayseian ridge is : %s" %long_term_score_br)
# print("long term score for caption linear regression is : %s" %long_term_score_lr)
# print("long term score for caption gradient boost is : %s" %long_term_score_gbr)
# print("long term score for caption random forest is : %s" %long_term_score_rfr)
# print("long term score for caption stacking regressor is : %s" %long_term_score_str)
# print("long term score for caption ridge linear regressor is : %s" %long_term_score_rlr)

# print("short term score for caption bayseian ridge is : %s" %short_term_score_br)
# print("short term score for caption linear regression is : %s" %short_term_score_lr)
# print("short term score for caption gradient boost is : %s" %short_term_score_gbr)
# print("short term score for caption random forest is : %s" %short_term_score_rfr)
# print("short term score for caption stacking regressor is : %s" %short_term_score_str)
# print("short term score for caption ridge linear regressor is : %s" %short_term_score_rlr)

# Note: Individual models below were run on several copies of the same notebook side by side and the results obtained from them are put below each of them 

### 1) running the bayseian ridge model for captions

In [ ]:
#extracting the caption feature 
caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#long term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
long_term_score_br = spearman_corr(BayesianRidge(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(long_term_score_br)

#short term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
short_term_score_br = spearman_corr(BayesianRidge(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(short_term_score_br)

### 2) running the linear regression model for captions

In [71]:
#extracting the caption feature 
caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#long term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
long_term_score_lr = spearman_corr(LinearRegression(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
#print(long_term_score_lr)

#short term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
short_term_score_lr = spearman_corr(LinearRegression(), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
#print(short_term_score_lr)

### 3) running the GradientBoost regression model for captions

In [ ]:
#extracting the caption feature 
caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#long term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
long_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(long_term_score_gbr)

#short term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
short_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(short_term_score_gbr)

### 4) running the Random forest regression model for captions

In [ ]:
#extracting the caption feature 
caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#long term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
long_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=100), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(long_term_score_rfr)

#short term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
short_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=100), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(short_term_score_rfr)

### 5) running the stacking regressor model for captions

In [ ]:
#extracting the caption feature 
caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#base estimators for the stacking regressor model with the fianl estimator being Linear regression
estimators = [('gbr', GradientBoostingRegressor(random_state=1, n_estimators=10)),('rfr', RandomForestRegressor(random_state=1, n_estimators=10)),('br', BayesianRidge())]

#long term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
long_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(long_term_score_str)

#short term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
short_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(short_term_score_str)

### 6) running the ridge linear regression model for captions

In [ ]:
#extracting the caption feature 
caption_features = training_set[training_set.columns[training_set.columns.str.startswith('caption')]]

#long term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_long_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
long_term_score_rlr = spearman_corr(Ridge(alpha=0.5), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(long_term_score_rlr)

#short term
X_train, X_test, y_train, y_test = train_test_split(caption_features,testing_set_short_term, test_size=0.2, random_state=1)
training_caption, testing_caption  = vectorize_data(X_train, X_test)
short_term_score_rlr = spearman_corr(Ridge(alpha=0.5), training_caption.toarray(), testing_caption.toarray(), y_train, y_test)
print(short_term_score_rlr)

# Spearman correlation values for long term and short term for captions feature

### we see that for captions feature bayseian regression model and stacking regressor has given similar results,we will choose bayseian ridge regressor as the predictor model for caption feature. Though, stacking regressor has given good results compared to bayseian, we choose bayseian as the model since the ensembled model worked out below did not yeild good overall results.

# working on c3d feature alone

In [17]:
#calculationg scores for long term and short term scores for C3D feature using regression models of sklearn like  
#bayseian ridge 
#linear regression 
#GradientBoostingRegressor
#RandomForestRegressor
#stackingRegressor (ensemble model with base estimators and one final estimator)
#Ridge regressor

#c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

#base estimators for the stacking regressor model with the fianl estimator being Linear regression
#estimators = [('gbr', GradientBoostingRegressor(random_state=1, n_estimators=10)),('rfr', RandomForestRegressor(random_state=1, n_estimators=10)),('br', BayesianRidge())]

#long_term
#X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)

# long_term_score_br = spearman_corr(BayesianRidge(), X_train, X_test, y_train, y_test)

# long_term_score_lr = spearman_corr(LinearRegression(), X_train, X_test, y_train, y_test)

# long_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)

# long_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)

# long_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), X_train, X_test, y_train, y_test)


# #short_term
# X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)

# short_term_score_br = spearman_corr(BayesianRidge(), X_train, X_test, y_train, y_test)

# short_term_score_lr = spearman_corr(LinearRegression(), X_train, X_test, y_train, y_test)

# short_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)

# short_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)

# short_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), X_train, X_test, y_train, y_test)

# short_term_score_rlr = spearman_corr(Ridge(alpha=0.5), X_train, X_test, y_train, y_test)

# print("long term score for C3D bayseian ridge is : %s" %long_term_score_br)
# print("long term score for C3D linear regression is : %s" %long_term_score_lr)
# print("long term score for C3D gradient boost is : %s" %long_term_score_gbr)
# print("long term score for C3D random forest is : %s" %long_term_score_rfr)
# print("long term score for C3D stacking regressor is : %s" %long_term_score_str)

# print("short term score for C3D bayseian ridge is : %s" %short_term_score_br)
# print("short term score for C3D linear regression is : %s" %short_term_score_lr)
# print("short term score for C3D gradient boost is : %s" %short_term_score_gbr)
# print("short term score for C3D random forest is : %s" %short_term_score_rfr)
# print("short term score for C3D stacking regressor is : %s" %short_term_score_str)


long term score for C3D bayseian ridge is : 0.1422393776581671
long term score for C3D linear regression is : 0.13934188580186976
long term score for C3D gradient boost is : 0.0965534778477242
long term score for C3D random forest is : 0.16218945295739265
long term score for C3D stacking regressor is : 0.17052570303114997
short term score for C3D bayseian ridge is : 0.3468355735922127
short term score for C3D linear regression is : 0.33542580349223455
short term score for C3D gradient boost is : 0.2989157017447577
short term score for C3D random forest is : 0.3838373993592676
short term score for C3D stacking regressor is : 0.37827702612132774


### 1) running the bayseian ridge model for C3D feature

In [ ]:
#extracting the C3D featute
c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

#long_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)
long_term_score_br = spearman_corr(BayesianRidge(), X_train, X_test, y_train, y_test)
print(long_term_score_br)

#short_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)
short_term_score_br = spearman_corr(BayesianRidge(), X_train, X_test, y_train, y_test)
print(short_term_score_br)

### 2) running the linear regression model for C3D feature

In [ ]:
#extracting the C3D featute
c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

#long_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)
long_term_score_lr = spearman_corr(LinearRegression(), X_train, X_test, y_train, y_test)
print(long_term_score_lr)

#short_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)
short_term_score_lr = spearman_corr(LinearRegression(), X_train, X_test, y_train, y_test)
print(short_term_score_lr)

### 3) running the gradient boost regressor for C3D feature

In [ ]:
#extracting the C3D featute
c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

#long_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)
long_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)
print(long_term_score_gbr)

#short_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)
short_term_score_gbr = spearman_corr(GradientBoostingRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)
print(short_term_score_gbr)

### 4) running the random forest regressor for C3D feature

In [ ]:
#extracting the C3D featute
c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

#long_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)
long_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)
print(long_term_score_rfr)

#short_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)
short_term_score_rfr = spearman_corr(RandomForestRegressor(random_state=1, n_estimators=500), X_train, X_test, y_train, y_test)
print(short_term_score_rfr)

### 5) running the stacking regressor for C3D feature

In [ ]:
#extracting the C3D featute
c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

#base estimators for the stacking regressor model with the fianl estimator being Linear regression
estimators = [('gbr', GradientBoostingRegressor(random_state=1, n_estimators=10)),('rfr', RandomForestRegressor(random_state=1, n_estimators=10)),('br', BayesianRidge())]

#long_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)
long_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), X_train, X_test, y_train, y_test)
print(long_term_score_str)

#short_term
X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)
short_term_score_str = spearman_corr(StackingRegressor(estimators=estimators,final_estimator=LinearRegression()), X_train, X_test, y_train, y_test)
print(long_term_score_str)

### 6) running the ridge linear regressor model for C3D feature

In [106]:
#extracting the C3D featute
c3d_features = training_set[training_set.columns[training_set.columns.str.startswith('C3D')]]

X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_long_term, test_size=0.2, random_state=1)
long_term_score_rlr = spearman_corr(Ridge(alpha=0.5), X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = train_test_split(c3d_features,testing_set_short_term, test_size=0.2, random_state=1)
short_term_score_rlr = spearman_corr(Ridge(alpha=0.5), X_train, X_test, y_train, y_test)

print("long term score for C3D ridge linear regressor is : %s" %long_term_score_rlr)
print("short term score for C3D ridge linear regressor is : %s" %short_term_score_rlr)

long term score for C3D ridge linear regressor is : 0.13398789781777398
short term score for C3D ridge linear regressor is : 0.34515321874559857


## Spearman correlation values for long term and short term for C3D feature

### we see that the stacking regressor model and random regressor have given similar results for long term scores and short term scores, choosing stacking regressor as the model for working with c3d feature on the test set.

# ensemble approach -1 (2 layered model)

### In the ensemble approach we will run the best selected models from above on the selected features 'C3D and Captions'

In [40]:
#getting the spearman correlation for the ensembled approach
def get_spearman_correlation(Predictions,y_test):
    corr, p_value = spearmanr(y_test, Predictions)
    return corr

In [55]:
def vectorize_one(X_training, X_validation, X_test):
    bow = TfidfVectorizer(lowercase=True, ngram_range=(1,2),analyzer = "word")
    train_bow = bow.fit(X_training['caption'])
    training_caption = train_bow.transform(X_training['caption'])
    validation_caption = train_bow.transform(X_validation['caption'])
    test_caption = train_bow.transform(X_test['caption'])
    return training_caption, validation_caption, test_caption

In [78]:
#setting up a bayseian ridge regression model(first level) for caption feature
def captions_model_one(X_training, X_validation, y_training, y_validation, X_test):
    training_caption, validation_caption, test_caption = vectorize_one(X_training[['caption']], X_validation[['caption']], X_test[['caption']])
    model = BayesianRidge()
    model.fit(training_caption.toarray(), y_training)
    predictions_validation = model.predict(validation_caption.toarray())
    predictions_test = model.predict(test_caption.toarray())
    return predictions_validation, predictions_test

#setting up a stacking regressor model (first level) with base estimators and final estimators for c3d feature
def C3D_model_one(X_training, X_validation, y_training, y_validation, X_test):
    C3D_features_training = X_training[X_training.columns[X_training.columns.str.startswith('C3D')]]
    C3D_features_validation = X_validation[X_validation.columns[X_validation.columns.str.startswith('C3D')]]
    C3D_features_test = X_test[X_test.columns[X_test.columns.str.startswith('C3D')]]
    #these are the estimators that are being used
    estimators = [('gbr', GradientBoostingRegressor(random_state=1, n_estimators=500)),('rfr', RandomForestRegressor(random_state=1, n_estimators=500)),('br', BayesianRidge())]
    model = StackingRegressor(estimators=estimators,final_estimator=LinearRegression())
    model.fit(C3D_features_training, y_training)
    predictions_validation = model.predict(C3D_features_validation)
    predictions_test = model.predict(C3D_features_test)
    return predictions_validation, predictions_test

In [79]:
def predict_scores(X_train, X_test, y_train, y_test):
    X_training,X_validation,y_training,y_validation= train_test_split(X_train,y_train, test_size=0.3, random_state=1)
    
    predictions_for_captions_validation, predictions_for_captions_test = captions_model_one(X_training, X_validation, y_training, y_validation, X_test)
    predictions_for_captions_validation = pd.DataFrame(predictions_for_captions_validation)
    predictions_for_captions_test = pd.DataFrame(predictions_for_captions_test)
    
    predictions_for_C3D_validation, predictions_for_C3D_test = C3D_model_one(X_training, X_validation, y_training, y_validation, X_test)
    predictions_for_C3D_validation = pd.DataFrame(predictions_for_C3D_validation)
    predictions_for_C3D_test = pd.DataFrame(predictions_for_C3D_test)
    
    df_validation = pd.concat([predictions_for_captions_validation,predictions_for_C3D_validation], axis=1)
    df_test = pd.concat([predictions_for_captions_test, predictions_for_C3D_test], axis=1)
    
    model = BayesianRidge()
    model.fit(df_validation,y_validation)
    predictions = model.predict(df_test)
    return predictions


In [80]:
#running the 2 layered model
ensemble_features=training_set.drop(columns='video')

#long term scores
X_train, X_test, y_train, y_test = train_test_split(ensemble_features,testing_set_long_term, test_size=0.2, random_state=1)
prediction_lt = predict_scores(X_train, X_test, y_train, y_test)
long_term_score_ensemble = get_spearman_correlation(prediction_lt,y_test)
print("spearman score for long term for ensembled 2 layer model is %s" %long_term_score_ensemble)

#short term scores
X_train, X_test, y_train, y_test = train_test_split(ensemble_features,testing_set_short_term, test_size=0.2, random_state=1)
prediction_st = predict_scores(X_train, X_test, y_train, y_test)
short_term_score_ensemble = get_spearman_correlation(prediction_st,y_test)
print("spearman score for short term for ensembled 2 layer model is %s" %short_term_score_ensemble)

spearman score for long term for ensembled 2 layer model is 0.23388682373746478
spearman score for short term for ensembled 2 layer model is 0.44113809301301843


# ensemble approach-2 (Weighted model)

In [73]:
def captions_model_second(X_train, X_test, y_train, y_test):
    training_caption, testing_caption  = vectorize_data(X_train[['caption']], X_test[['caption']])
    model = BayesianRidge()
    model.fit(training_caption.toarray(), y_train)
    predictions_validation = model.predict(testing_caption.toarray())
    return predictions_validation

def C3D_model_second(X_train, X_test, y_train, y_test):
    C3D_features_train = X_train[X_train.columns[X_train.columns.str.startswith('C3D')]]
    C3D_features_test = X_test[X_test.columns[X_test.columns.str.startswith('C3D')]]
    #these are the estimators that are being used
    estimators = [('gbr', GradientBoostingRegressor(random_state=1, n_estimators=500)),('rfr', RandomForestRegressor(random_state=1, n_estimators=500)),('br', BayesianRidge())]
    model = StackingRegressor(estimators=estimators,final_estimator=LinearRegression())
    model.fit(C3D_features_train, y_train)
    predictions_test = model.predict(C3D_features_test)
    return predictions_test

In [74]:
def predict_long_term(X_train, X_test, y_train, y_test):
    predictions_for_captions_long_term = captions_model_second(X_train, X_test, y_train, y_test)
    predictions_for_C3D_long_term = C3D_model_second(X_train, X_test, y_train, y_test)
    Prediction_final = 0.5*predictions_for_captions_long_term + 0.5*predictions_for_C3D_long_term
    return Prediction_final

In [75]:
def predict_short_term(X_train, X_test, y_train, y_test):
    predictions_for_captions_short_term = captions_model_second(X_train, X_test, y_train, y_test)
    predictions_for_C3D_short_term = C3D_model_second(X_train, X_test, y_train, y_test)
    Prediction_final = 0.6*predictions_for_captions_short_term + 0.4*predictions_for_C3D_short_term
    return Prediction_final

In [76]:
#running the weighted model
ensemble_features=training_set.drop(columns='video')

#long term scores
X_train, X_test, y_train, y_test = train_test_split(ensemble_features,testing_set_long_term, test_size=0.2, random_state=1)
prediction_lt = predict_long_term(X_train, X_test, y_train, y_test)
long_term_score_ensemble = get_spearman_correlation(prediction_lt,y_test)
print("spearman score for long term for ensembled weighted model is %s" %long_term_score_ensemble)

#short term scores
X_train, X_test, y_train, y_test = train_test_split(ensemble_features,testing_set_short_term, test_size=0.2, random_state=1)
prediction_st = predict_short_term(X_train, X_test, y_train, y_test)
short_term_score_ensemble = get_spearman_correlation(prediction_st,y_test)
print("spearman score for short term for ensembled weighted model is %s" %short_term_score_ensemble)

spearman score for long term for ensembled weighted model is 0.24189099108263637
spearman score for short term for ensembled weighted model is 0.45484633969047794


## Note on how the ensemble model is selected for test data

# working on test data

In [83]:
os.getcwd()
os.chdir(r'C:\Users\navee\Desktop')
os.getcwd()

'C:\\Users\\navee\\Desktop'

In [84]:
#read captions
dataframe_captions_test = pd.read_csv(r'./Test-set/Captions_test/test-set-1_video-captions.txt', sep='\t',names = ['video','caption'])
dataframe_captions_test.head()

,video,caption
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


In [85]:
#read c3d test data
path='./Test-set/C3D_test'
files = reading_file_names(path)
dataframe_C3D_test = pd.DataFrame()
for filename in files:
  #print(filename)
  dataframe_C3D_test = c3d_read(dataframe_C3D_test, filename)
dataframe_C3D_test.head()

total files read = 2000


,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,video
0,0.017933,0.017731,3.270400e-03,3.460000e-06,1.188000e-05,9.952000e-05,1.680540e-03,2.201350e-03,2.255180e-03,8.569500e-04,...,1.085400e-04,0.006845,2.773800e-04,6.760000e-06,2.205000e-05,0.000614,0.129894,0.151878,NaN,video10000.webm
1,0.023967,0.001808,1.783000e-05,6.000000e-08,1.000000e-08,0.000000e+00,1.100000e-07,1.000000e-08,4.700000e-07,1.500000e-07,...,0.000000e+00,0.000059,1.000000e-08,1.000000e-08,2.800000e-07,0.000021,0.001365,0.000015,NaN,video10001.webm
2,0.018696,0.862584,5.000000e-07,1.120000e-06,1.460000e-06,9.000000e-08,5.770000e-06,7.769000e-05,8.540000e-06,3.570000e-06,...,3.000000e-08,0.000021,2.300000e-07,1.400000e-07,2.020000e-06,0.000019,0.020344,0.000105,NaN,video10002.webm
3,0.000556,0.000248,2.400031e-01,3.271000e-05,9.800000e-07,4.246000e-05,2.813900e-04,8.360000e-06,8.720000e-05,4.609900e-04,...,1.237120e-03,0.000010,1.118000e-05,1.036200e-04,3.430500e-03,0.029655,0.000314,0.000372,NaN,video10003.webm
4,0.050587,0.006240,1.170256e-01,5.364600e-04,1.415000e-05,2.702100e-04,1.699570e-03,2.862500e-04,2.010600e-04,8.166000e-05,...,6.122000e-05,0.001240,2.447000e-05,4.139000e-05,5.321400e-04,0.000371,0.005023,0.051066,NaN,video10004.webm


In [86]:
#drop NA values and predix each c3d column with 'c3d'
dataframe_C3D_test = dataframe_C3D_test.drop(columns=101)

In [87]:
new_columns = list()
for colname in dataframe_C3D_test.columns:
    new_columns.append('C3D_' + str(colname))
dataframe_C3D_test.columns = new_columns
dataframe_C3D_test = dataframe_C3D_test.rename(columns={'C3D_video':'video'})
dataframe_C3D_test.head()

,C3D_0,C3D_1,C3D_2,C3D_3,C3D_4,C3D_5,C3D_6,C3D_7,C3D_8,C3D_9,...,C3D_92,C3D_93,C3D_94,C3D_95,C3D_96,C3D_97,C3D_98,C3D_99,C3D_100,video
0,0.017933,0.017731,3.270400e-03,3.460000e-06,1.188000e-05,9.952000e-05,1.680540e-03,2.201350e-03,2.255180e-03,8.569500e-04,...,4.524400e-04,1.085400e-04,0.006845,2.773800e-04,6.760000e-06,2.205000e-05,0.000614,0.129894,0.151878,video10000.webm
1,0.023967,0.001808,1.783000e-05,6.000000e-08,1.000000e-08,0.000000e+00,1.100000e-07,1.000000e-08,4.700000e-07,1.500000e-07,...,0.000000e+00,0.000000e+00,0.000059,1.000000e-08,1.000000e-08,2.800000e-07,0.000021,0.001365,0.000015,video10001.webm
2,0.018696,0.862584,5.000000e-07,1.120000e-06,1.460000e-06,9.000000e-08,5.770000e-06,7.769000e-05,8.540000e-06,3.570000e-06,...,2.300000e-07,3.000000e-08,0.000021,2.300000e-07,1.400000e-07,2.020000e-06,0.000019,0.020344,0.000105,video10002.webm
3,0.000556,0.000248,2.400031e-01,3.271000e-05,9.800000e-07,4.246000e-05,2.813900e-04,8.360000e-06,8.720000e-05,4.609900e-04,...,1.452000e-04,1.237120e-03,0.000010,1.118000e-05,1.036200e-04,3.430500e-03,0.029655,0.000314,0.000372,video10003.webm
4,0.050587,0.006240,1.170256e-01,5.364600e-04,1.415000e-05,2.702100e-04,1.699570e-03,2.862500e-04,2.010600e-04,8.166000e-05,...,2.231000e-05,6.122000e-05,0.001240,2.447000e-05,4.139000e-05,5.321400e-04,0.000371,0.005023,0.051066,video10004.webm


In [88]:
caption_c3d_merge_test = dataframe_captions_test.merge(dataframe_C3D_test)
caption_c3d_merge_test.head()

,video,caption,C3D_0,C3D_1,C3D_2,C3D_3,C3D_4,C3D_5,C3D_6,C3D_7,...,C3D_91,C3D_92,C3D_93,C3D_94,C3D_95,C3D_96,C3D_97,C3D_98,C3D_99,C3D_100
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks,0.000136,0.000213,0.001248,0.000011,1.180000e-06,4.276000e-05,5.800000e-07,5.820000e-06,...,0.000003,5.620000e-06,1.260000e-06,0.000199,1.500000e-06,2.000000e-07,5.840000e-06,0.000006,0.000210,0.195120
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...,0.000127,0.000024,0.049501,0.000024,2.361000e-05,3.440000e-06,5.102000e-05,1.240200e-04,...,0.000173,3.019200e-04,2.649800e-04,0.000163,1.050000e-05,2.540000e-06,3.619300e-04,0.000016,0.000010,0.000392
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...,0.488982,0.000585,0.000025,0.000160,2.400000e-07,8.300000e-07,3.620000e-06,5.200000e-07,...,0.000002,1.900000e-07,3.200000e-07,0.001802,2.640000e-06,9.000000e-08,2.600000e-07,0.000067,0.000123,0.000015
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...,0.249251,0.001018,0.000008,0.000019,2.200000e-07,8.800000e-07,1.646560e-02,4.701000e-05,...,0.000065,8.400000e-07,1.700000e-07,0.000808,9.700000e-07,7.670000e-06,9.100000e-07,0.000015,0.000707,0.000010
4,video7498.webm,slow-motion-of-photographer-in-national-park,0.009894,0.000034,0.002613,0.000002,1.010000e-06,8.770000e-06,2.610000e-06,4.500000e-07,...,0.000012,6.600000e-07,8.770000e-06,0.000132,7.400000e-07,2.120000e-06,8.260000e-06,0.001402,0.000745,0.000456


In [89]:
#reading ground truth values for test set
ground_truth_test = pd.read_csv(r'./Test-set/Ground-truth_test/ground_truth_template.csv')
ground_truth_test.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


In [90]:
#the video names are not in the format like in the caption_c3d_merge_test data frame

#function to change the video name in dataframe_ground_truth dataframe
def change_video_name(val):
  val = 'video' +str(val) + '.webm' 
  return val

#function to remove the video number and retain only the number, used when populating the final csv
def change_video_name_remove(val):
  val = str(val)[str(val).find('o')+1:str(val).find('.')]
  return val

In [91]:
ground_truth_test['video'] = ground_truth_test['video'].apply(change_video_name)
ground_truth_test.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video7494.webm,NaN,33,NaN,12
1,video7495.webm,NaN,34,NaN,10
2,video7496.webm,NaN,32,NaN,13
3,video7497.webm,NaN,33,NaN,10
4,video7498.webm,NaN,33,NaN,10


In [92]:
#merging the dataframes of features of the test set
gt_caption_c3d_merge_test = caption_c3d_merge_test.merge(ground_truth_test)
gt_caption_c3d_merge_test.head()

,video,caption,C3D_0,C3D_1,C3D_2,C3D_3,C3D_4,C3D_5,C3D_6,C3D_7,...,C3D_95,C3D_96,C3D_97,C3D_98,C3D_99,C3D_100,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks,0.000136,0.000213,0.001248,0.000011,1.180000e-06,4.276000e-05,5.800000e-07,5.820000e-06,...,1.500000e-06,2.000000e-07,5.840000e-06,0.000006,0.000210,0.195120,NaN,33,NaN,12
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...,0.000127,0.000024,0.049501,0.000024,2.361000e-05,3.440000e-06,5.102000e-05,1.240200e-04,...,1.050000e-05,2.540000e-06,3.619300e-04,0.000016,0.000010,0.000392,NaN,34,NaN,10
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...,0.488982,0.000585,0.000025,0.000160,2.400000e-07,8.300000e-07,3.620000e-06,5.200000e-07,...,2.640000e-06,9.000000e-08,2.600000e-07,0.000067,0.000123,0.000015,NaN,32,NaN,13
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...,0.249251,0.001018,0.000008,0.000019,2.200000e-07,8.800000e-07,1.646560e-02,4.701000e-05,...,9.700000e-07,7.670000e-06,9.100000e-07,0.000015,0.000707,0.000010,NaN,33,NaN,10
4,video7498.webm,slow-motion-of-photographer-in-national-park,0.009894,0.000034,0.002613,0.000002,1.010000e-06,8.770000e-06,2.610000e-06,4.500000e-07,...,7.400000e-07,2.120000e-06,8.260000e-06,0.001402,0.000745,0.000456,NaN,33,NaN,10


In [93]:
#cleaning the caption data of test set.

#Converting the captions into lowercase
gt_caption_c3d_merge_test['caption'] = gt_caption_c3d_merge_test['caption'].str.replace('-',' ')

#Removal of punctuations 
gt_caption_c3d_merge_test['caption'] = gt_caption_c3d_merge_test['caption'].str.replace('[^\w\s]','')

#Removal of stop words
words = stopwords.words('english')
gt_caption_c3d_merge_test['caption'] = gt_caption_c3d_merge_test['caption'].apply(lambda x: " ".join(each for each in x.split() if each not in words))
#gt_caption_c3d_merge['caption'].head()

#Lemmatizing our captions 
lemmatizer = WordNetLemmatizer() 
gt_caption_c3d_merge_test['caption'] = gt_caption_c3d_merge_test['caption'].apply(lambda x: " ".join([lemmatizer.lemmatize(each) for each in x.split()]))
#gt_caption_c3d_merge['caption'].head()

#removing digits from captions
gt_caption_c3d_merge_test['caption'] = gt_caption_c3d_merge_test['caption'].str.replace('[0-9]','')
gt_caption_c3d_merge_test.head()

,video,caption,C3D_0,C3D_1,C3D_2,C3D_3,C3D_4,C3D_5,C3D_6,C3D_7,...,C3D_95,C3D_96,C3D_97,C3D_98,C3D_99,C3D_100,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video7494.webm,green jeep struggling drive huge rock,0.000136,0.000213,0.001248,0.000011,1.180000e-06,4.276000e-05,5.800000e-07,5.820000e-06,...,1.500000e-06,2.000000e-07,5.840000e-06,0.000006,0.000210,0.195120,NaN,33,NaN,12
1,video7495.webm,hiking woman tourist walking forward mountain ...,0.000127,0.000024,0.049501,0.000024,2.361000e-05,3.440000e-06,5.102000e-05,1.240200e-04,...,1.050000e-05,2.540000e-06,3.619300e-04,0.000016,0.000010,0.000392,NaN,34,NaN,10
2,video7496.webm,close african american doctor hand using sphyg...,0.488982,0.000585,0.000025,0.000160,2.400000e-07,8.300000e-07,3.620000e-06,5.200000e-07,...,2.640000e-06,9.000000e-08,2.600000e-07,0.000067,0.000123,0.000015,NaN,32,NaN,13
3,video7497.webm,slow motion man using treadmill gym regular ph...,0.249251,0.001018,0.000008,0.000019,2.200000e-07,8.800000e-07,1.646560e-02,4.701000e-05,...,9.700000e-07,7.670000e-06,9.100000e-07,0.000015,0.000707,0.000010,NaN,33,NaN,10
4,video7498.webm,slow motion photographer national park,0.009894,0.000034,0.002613,0.000002,1.010000e-06,8.770000e-06,2.610000e-06,4.500000e-07,...,7.400000e-07,2.120000e-06,8.260000e-06,0.001402,0.000745,0.000456,NaN,33,NaN,10


building the model for the test set

In [94]:
X_training_set =  gt_caption_c3d_merge.drop(columns=['short-term_memorability','long-term_memorability','video'])
Y_training_set = gt_caption_c3d_merge[['short-term_memorability', 'long-term_memorability']]

In [95]:
X_testing_set = gt_caption_c3d_merge_test.drop(columns=['short-term_memorability','long-term_memorability','nb_short-term_annotations','nb_long-term_annotations','video'])
Y_testing_set = gt_caption_c3d_merge_test[['short-term_memorability','long-term_memorability']]

In [96]:
#data frame to capture the predicted values
Predictions = pd.DataFrame()
Predictions['video'] = gt_caption_c3d_merge_test['video']

In [97]:
print(X_testing_set.shape)
print(Y_testing_set.shape)

(2000, 102)
(2000, 2)


In [98]:
#Applying model
X_train, X_test, y_train, y_test = X_training_set, X_testing_set, Y_training_set['long-term_memorability'],Y_testing_set['long-term_memorability']
Predictions['long-term_memorability'] = predict_long_term(X_train, X_test, y_train, y_test)
X_train, X_test, y_train, y_test = X_training_set, X_testing_set, Y_training_set['short-term_memorability'],Y_testing_set['short-term_memorability']
Predictions['short-term_memorability'] = predict_short_term(X_train, X_test, y_train, y_test)


In [99]:
avg_score_short_term_test_data = sum(Predictions['short-term_memorability'])/len(Predictions['short-term_memorability'])
print("average short term score test = " + str(avg_score_short_term_test_data))

avg_score_long_term_test_data = sum(Predictions['long-term_memorability'])/len(Predictions['long-term_memorability'])
print("average long term score test = " + str(avg_score_long_term_test_data))

average short term score test = 0.8589663259124821
average long term score test = 0.7782296832220857


In [100]:
#removing video prefix and suffix to match with the ground truth template
Predictions['video'] = Predictions['video'].apply(change_video_name_remove)
ground_truth_test['video'] = ground_truth_test['video'].apply(change_video_name_remove)

In [101]:
#dropping the long-term and short-term memorability scores to accomodate the predicted scores
ground_truth_test = ground_truth_test.drop(columns=['long-term_memorability','short-term_memorability'])

In [102]:
#merging the predictions dataframe to fill in the new predicted values
ground_truth_test = ground_truth_test.merge(Predictions)

In [103]:
#arranging xolumns as per the format
ground_truth_test = ground_truth_test[['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations']]

In [104]:
#saving the results to the csv
ground_truth_test.to_csv('ground_truth_results.csv')